In [9]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init
import torch.utils.data as data
import torchvision.datasets as dset
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torch.autograd import Variable

In [89]:
transform = transforms.Compose([transforms.CenterCrop(224),transforms.RandomHorizontalFlip(),transforms.ToTensor(),transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])

In [90]:
trainset = dset.CIFAR10('./',train =True, download = True,transform = transform)
testset  = dset.CIFAR10('./',train =False, download = True, transform =transform)

Files already downloaded and verified
Files already downloaded and verified


In [114]:
train_loader = DataLoader(trainset, batch_size =2, shuffle = True, num_workers=2)
test_loader =DataLoader(testset, batch_size=2, shuffle=False, num_workers=2)

In [109]:
def conv1(in_dim, out_dim):
    model = nn.Sequential(
        nn.Conv2d(in_dim,out_dim,1,1),
        nn.ReLU(),
    )
    return model

def conv3(in_dim, mid_dim,out_dim):
    model = nn.Sequential(
        nn.Conv2d(in_dim,mid_dim,1,1),
        nn.Conv2d(mid_dim,out_dim,3,1,1), # fiter / stride / padding
        nn.ReLU()
    )
    return model

def conv5(in_dim, mid_dim, out_dim):
    model = nn.Sequential(
        nn.Conv2d(in_dim,mid_dim,1,1),
        nn.Conv2d(mid_dim,out_dim,5,1,2),
        nn.ReLU()        
    )
    return model

def max(in_dim,out_dim):
    model = nn.Sequential(
        nn.MaxPool2d(3,1,1),
        nn.Conv2d(in_dim,out_dim,1,1),
        nn.ReLU()
    )
    return model

In [110]:
class inception(nn.Module):
    def __init__(self,in_dim,out_dim1, mid_dim3,out_dim3, mid_dim5, out_dim5,out_dim_max):
        super(inception,self).__init__()
        self.conv1 = conv1(in_dim, out_dim1)
        self.conv3 = conv3(in_dim, mid_dim3,out_dim3)
        self.conv5 = conv5(in_dim, mid_dim5, out_dim5)
        self.max = max(in_dim,out_dim_max)
        
    def forward(self,x):
        output1 = self.conv1(x)
        output3 = self.conv3(x)
        output5 = self.conv5(x)
        outputm = self.max(x)
        output = torch.cat([output1,output3,output5,outputm],1)
        
        return output
        

In [127]:
class GoogLeNet(nn.Module):
    
#in_dim,out_dim1, mid_dim3,out_dim3, mid_dim5, out_dim5,out_dim_max

    def __init__(self, base_dim, num_classes=2):
        super(GoogLeNet, self).__init__()
        self.layer_1 = nn.Sequential(
            nn.Conv2d(3,base_dim,7,2,3),
            nn.MaxPool2d(3,2,1),
            nn.Conv2d(base_dim,base_dim*3,3,1,1),
            nn.MaxPool2d(3,2,1),
        )
        self.layer_2 = nn.Sequential(
            inception(base_dim*3,64,96,128,16,32,32),
            inception(base_dim*4,128,128,192,32,96,64),
            nn.MaxPool2d(3,2,1),
        )
        self.layer_3 = nn.Sequential(
            inception(480,192,96,208,16,48,64),
            inception(512,160,112,224,24,64,64),
            inception(512,128,128,256,24,64,64),
            inception(512,112,144,288,32,64,64),
            inception(528,256,160,320,32,128,128),
            nn.MaxPool2d(3,2,1),
        )
        self.layer_4 = nn.Sequential(
            inception(832,256,160,320,32,128,128),
            inception(832,384,192,384,48,128,128), 
            nn.AvgPool2d(7,1),
        )
        self.layer_5 = nn.Dropout2d(0.4)
        self.fc_layer = nn.Linear(1024,10)
                
        
    def forward(self, out):
        out = self.layer_1(out)
        out = self.layer_2(out)
        out = self.layer_3(out)
        out = self.layer_4(out)
        print(out.size())
        out = self.layer_5(out)
        print(out.size())
        out = out.view(out.size(0),-1)
        print(out.size(0))
        print(out.size())
        out = self.fc_layer(out)
        #print(out.size())
              
        return out
        
    
model = GoogLeNet(base_dim =64)

for i in model.named_children():
    print(i)

('layer_1', Sequential(
  (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
  (1): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (2): Conv2d(64, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
))
('layer_2', Sequential(
  (0): inception(
    (conv1): Sequential(
      (0): Conv2d(192, 64, kernel_size=(1, 1), stride=(1, 1))
      (1): ReLU()
    )
    (conv3): Sequential(
      (0): Conv2d(192, 96, kernel_size=(1, 1), stride=(1, 1))
      (1): Conv2d(96, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (2): ReLU()
    )
    (conv5): Sequential(
      (0): Conv2d(192, 16, kernel_size=(1, 1), stride=(1, 1))
      (1): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
      (2): ReLU()
    )
    (max): Sequential(
      (0): MaxPool2d(kernel_size=3, stride=1, padding=1, dilation=1, ceil_mode=False)
      (1): Conv2d(192, 3

In [128]:
loss_f = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(),lr = 0.002)

In [129]:
for i in range(100):
    for img, label in train_loader:
        img = Variable(img)
        label = Variable(label)
        
        optimizer.zero_grad()
        output = model(img)
        
        loss = loss_f(output,label)
        loss.backward()
        optimizer.step()
        print(loss)
        print("\n")
    if i%1 ==0:
        print(loss)
       
    
# drop out 을 해도 변하는건 없다
# out.size(0) 은 1번째 dimension
# .view(out.size(0),-1) 1번째 dimension 이외 모든 dimension 합쳐진다.

torch.Size([2, 1024, 1, 1])
torch.Size([2, 1024, 1, 1])
2
torch.Size([2, 1024])
tensor(2.3117, grad_fn=<NllLossBackward>)


torch.Size([2, 1024, 1, 1])
torch.Size([2, 1024, 1, 1])
2
torch.Size([2, 1024])
tensor(2.3724, grad_fn=<NllLossBackward>)


torch.Size([2, 1024, 1, 1])
torch.Size([2, 1024, 1, 1])
2
torch.Size([2, 1024])
tensor(2.2540, grad_fn=<NllLossBackward>)


torch.Size([2, 1024, 1, 1])
torch.Size([2, 1024, 1, 1])
2
torch.Size([2, 1024])
tensor(2.3653, grad_fn=<NllLossBackward>)


torch.Size([2, 1024, 1, 1])
torch.Size([2, 1024, 1, 1])
2
torch.Size([2, 1024])
tensor(2.6840, grad_fn=<NllLossBackward>)


torch.Size([2, 1024, 1, 1])
torch.Size([2, 1024, 1, 1])
2
torch.Size([2, 1024])
tensor(2.4275, grad_fn=<NllLossBackward>)


torch.Size([2, 1024, 1, 1])
torch.Size([2, 1024, 1, 1])
2
torch.Size([2, 1024])
tensor(2.3425, grad_fn=<NllLossBackward>)


torch.Size([2, 1024, 1, 1])
torch.Size([2, 1024, 1, 1])
2
torch.Size([2, 1024])
tensor(2.3080, grad_fn=<NllLossBackward>)


torch.Si

Traceback (most recent call last):
  File "/usr/lib/python3.5/multiprocessing/queues.py", line 247, in _feed
    send_bytes(obj)
  File "/usr/lib/python3.5/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/usr/lib/python3.5/multiprocessing/connection.py", line 404, in _send_bytes
    self._send(header + buf)
  File "/usr/lib/python3.5/multiprocessing/connection.py", line 368, in _send
    n = write(self._handle, buf)
BrokenPipeError: [Errno 32] Broken pipe
Traceback (most recent call last):
  File "/usr/lib/python3.5/multiprocessing/queues.py", line 247, in _feed
    send_bytes(obj)
  File "/usr/lib/python3.5/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/usr/lib/python3.5/multiprocessing/connection.py", line 404, in _send_bytes
    self._send(header + buf)
  File "/usr/lib/python3.5/multiprocessing/connection.py", line 368, in _send
    n = write(self._handle, b

KeyboardInterrupt: 